This is the notebook for experimenting with a zero-shot sliding-window Question-Answering model approach to the task of classifying documents based on whether or not they give evidence that the company submitting the document has provided any training about modern slavery to their employees. 

The motivation is to use transfer learning from models pre-trained to extract relevant answers (as a span) from a document (context) in order to automate the identification of which small subsets of the documents might be relevant to modern slavery training. These smaller subsets can then make the job of human-labelling additional documents more efficient or be fed into another model (perhaps a transformer trained for sequence classification) which can only handle a limited number of tokens.

The idea behind the approach is to use a pretrained QA model (one trained on SQuAD v2 such that it can return a "no span found" result) to ask questions of the documents. Since most documents in the dataset are longer than the maximum input length, a sliding window approach is used: after the entire document is tokenized, the QA model is run on successive windows, each slid by stride=128 tokens (~1/4th of the window size). All spans returned by the QA model are recorded in a new dataframe (df_with_segments.parquet). A notebook for visualizing the results of the sliding-window QA model approach is available: 'QA results viewer.ipynb'

Six questions are trialed to see which one(s) provide the best results:
 - 'Is there training provided?'
 - 'Is there training already in place?'
 - 'Has training been done?'
 - 'Is training planned?'
 - 'Is training in development?'
 - 'What kind of training is provided?'

Note: as this is a zero-shot approach, we can ignore the labels as we will not be doing any training. Therefore, the labeled (train) and unlabeled (test) data will be concatenated into a single input dataframe.

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import re
from datetime import datetime, timedelta

PyTorch version 1.6.0 available.
TensorFlow version 2.3.0 available.


In [2]:
# import the data, strip away the labels and combine into a single df
df_labeled=pd.read_csv('train (3).csv',index_col=0)
df_hidden=pd.read_csv('test (3).csv',index_col=0)
df_labeled['source']='labeled'
df_hidden['source']='hidden'
df = pd.concat([df_labeled[['source','TEXT']], 
                df_hidden[['source','TEXT']]],axis=0).reset_index()
df

,ID,source,TEXT
0,0,labeled,Modern Slavery Statement\n\nUa\n\n> Responsibi...
1,1,labeled,Burton's Biscuit Company (a trading name of Bu...
2,2,labeled,MODERN SLAVERY ACT STATEMENT\nOUR BUSINESS Zal...
3,3,labeled,MENU\nHOME\nU.K. MODERN SLAVERY ACT STATEMENT\...
4,4,labeled,Modern Slavery Act Statement\nIntroduction fro...
...,...,...,...
976,326,hidden,CECP Advisors LLP Modern Slavery Act Statement...
977,327,hidden,Modern Slavery Act Transparency Statement\n201...
978,328,hidden,MENU\n\n0333 2203 121\nBOOK A ROOM\n\nAnti Sla...
979,329,hidden,"We have placed cookies on your computer, as th..."


In [3]:
# any characters repeated more than 4 times will be shortened to 4 repetitions: 
# https://stackoverflow.com/questions/10072744/remove-repeating-characters-from-words
df['TEXT']=df['TEXT'].apply(lambda x: re.sub(r'(.)\1{4,}', r'\1\1\1\1', str(x)))

In [4]:
# Model chosen based on SQuAD v2 leaderboards December 2020
model_name = 'ktrapeznikov/albert-xlarge-v2-squad-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model.eval()

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/ktrapeznikov/albert-xlarge-v2-squad-v2/config.json from cache at C:\Users\dhilg/.cache\torch\transformers\c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f
Model config AlbertConfig {
  "architectures": [
    "AlbertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 16,
  "num_hidden_groups": 1,
  "num_hidden_layers": 24,
  "num_memory_blocks": 0,
  "output_past": true,
  "pad_toke

AlbertForQuestionAnswering(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=2048, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=2048, out_features=2048, bias=True)
                (key): Linear(in_features=2048, out_features=2048, bias=True)
                (value): Linear(in_features=2048, out_features=

In [5]:
def classify_tokens(model, tokenizer, questions, tokens, max_batch_size=8, max_model_tokens=512, stride=64):
    num_tokens = tokens.size()[1]
    token_classes = torch.zeros((len(questions), num_tokens), dtype=torch.long)

    batch_details=[]
    for i, question in enumerate(questions):
        question_tokens = tokenizer(question, return_tensors='pt')['input_ids']
        num_question_tokens = question_tokens.size()[1]
        max_context_tokens = max_model_tokens - num_question_tokens - 1 # -1 for the [SEP] token that will be added to the end

        num_windows = max(1, -(-(num_tokens - max_context_tokens) // stride) + 1)
        
        for j in range(num_windows):
            end = min(j*stride + max_context_tokens, num_tokens)
            start = max(0, end - max_context_tokens)
            batch_details.append({'question number':i,
                                  'question tokens':question_tokens,
                                  'token start':start,
                                  'token end':end
                                 })
            if len(batch_details) >= max_batch_size or (i==len(questions) and j==num_windows):
                token_classes = run_batch(model, tokens, token_classes, batch_details)
                batch_details = []
    
    return token_classes

def run_batch(model, tokens, token_classes, batch_details):
    inputs = batch_inputs(tokens, batch_details)
    
    answer_start_logits, answer_end_logits = model(**inputs)
    
    combined_logits = torch.cat([answer_start_logits.unsqueeze(0), answer_end_logits.unsqueeze(0)])
    span_input_ids = torch.max(combined_logits,2)[1]
    span_input_ids[1] += 1 # need to add 1 to end token of span
    
    # need to slide the token ids to remove the question tokens from the front of the tensor:
    input_question_lengths = torch.tensor([instance['question tokens'].size()[1] for instance in batch_details], 
                                          dtype=torch.long)
    span_token_ids = torch.max(span_input_ids - input_question_lengths, 
                               torch.zeros_like(span_input_ids)) # if no span is found, the model will point to the first 
                                                                 # token of the question. After subtracting the question
                                                                 # length, this would be negative, so set a floor of zero.
    for i, instance in enumerate(batch_details):
        # if a span was found, span_token_ids of the start token will be > 0
        if span_token_ids[0,i] > 0:
            span_token_start = span_token_ids[0,i] + instance['token start']
            span_token_end = span_token_ids[1,i] + instance['token start']
            
            token_classes[instance['question number'],span_token_start:span_token_end+1] = 1
    
    return token_classes
    
def batch_inputs(tokens, batch_details):
    max_tokens = max_batch_tokens(batch_details)
    batch_size = len(batch_details)
    
    inputs={'input_ids':torch.zeros(batch_size, max_tokens, dtype=torch.long),
            'token_type_ids':torch.zeros(batch_size, max_tokens, dtype=torch.long),
            'attention_mask':torch.zeros(batch_size, max_tokens, dtype=torch.long)
           }
    
    for i, instance in enumerate(batch_details):
        question_tokens = instance['question tokens']
        context_tokens = tokens[:, instance['token start']:instance['token end']]
        
        question_length = question_tokens.size()[1]
        context_length = context_tokens.size()[1]
        
        inputs['input_ids'][i, :question_length]=question_tokens.squeeze(0)
        inputs['input_ids'][i, question_length:question_length+context_length]=context_tokens.squeeze(0)
        # add final [SEP] token after context (same as final token of question tokens)
        inputs['input_ids'][i, -1]=question_tokens.squeeze(0)[-1]
        
        inputs['token_type_ids'][i, question_length:] = 1
        inputs['attention_mask'][i, :question_length+context_length+1] = 1
    return inputs

def max_batch_tokens(batch_details):
    max_tokens = 0
    for instance in batch_details:
        num_question_tokens = instance['question tokens'].size()[1] - 2 # don't count the [CLS] and [SEP] tokens
        num_context_tokens = instance['token end'] - instance['token start']
        instance_tokens = 1 + num_question_tokens + 1 + num_context_tokens + 1
        if instance_tokens > max_tokens:
            max_tokens = instance_tokens
    return max_tokens

def process_text(text, model, tokenizer, questions, max_batch_size=8, max_model_tokens=512, stride=64):
    tokens = tokenizer(text, return_tensors='pt')['input_ids'][:,1:-1] # drop first and last tokens ([CLS] and [SEP])
    
    token_classes = classify_tokens(model, tokenizer, questions, tokens, max_batch_size, max_model_tokens, stride)
    
    filtered_text=[]
    for i, question in enumerate(questions):
        filtered_text.append({'question':question,
                              'text segments':token_classes_to_str(tokenizer, tokens, token_classes[i])
                             })
    
    return filtered_text

def token_classes_to_str(tokenizer, tokens, token_classes):
    spans = identify_distinct_spans(token_classes)
    
    text_segments = []
    for span_start, span_end in spans:
        text_segments.append(tokens_to_str(tokenizer, tokens, span_start, span_end))
        
    return text_segments

def tokens_to_str(tokenizer, tokens, span_start, span_end):
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens[0][span_start:span_end+1]))
        
def identify_distinct_spans(token_classes):
    spans=[]
    last_token_class=0
    for i in range(token_classes.size()[0]):
        curr_token_class = token_classes[i]
        if last_token_class != curr_token_class:
            if last_token_class == 0:
                span_start = i
            else:
                span_end = i-1
                spans.append((span_start, span_end))
        last_token_class = curr_token_class
    
    return spans

In [6]:
def process_row(row_id, model, tokenizer, questions, max_batch_size=8, max_model_tokens=512, stride=64):
    filtered_text = process_text(text=df.loc[row_id,'TEXT'], 
                                 model=model, 
                                 tokenizer=tokenizer, 
                                 questions=questions, 
                                 max_batch_size=max_batch_size, 
                                 max_model_tokens=max_model_tokens, 
                                 stride=stride)
    for question in filtered_text:
        col_header = question['question']
        text_segments = question['text segments']
        for text_segment in text_segments:
            df.loc[row_id,col_header].append(text_segment)

In [7]:
questions=['Is there training provided?', 
           'Is there training already in place?',
           'Has training been done?',
           'Is training planned?',
           'Is training in development?',
           'What kind of training is provided?'
          ]

for question in questions:
    df[question]=[[] for _ in range(len(df))]

In [ ]:
start_time=datetime.now()

for i in range(df.shape[0]):
    row_start = datetime.now()
    process_row(row_id=i, 
                model=model, 
                tokenizer=tokenizer, 
                questions=questions, 
                max_batch_size=2, 
                max_model_tokens=512, 
                stride=128)
    df.to_parquet('df_with_segments.parquet')
    print(f'row {i} finished at {datetime.now()}. Row time = {datetime.now() - row_start}. Total time elapsed = {datetime.now() - start_time}')
    
df

Token indices sequence length is longer than the specified maximum sequence length for this model (870 > 512). Running this sequence through the model will result in indexing errors


row 0 finished at 2020-12-11 13:23:36.092920. Row time = 0:04:39.406075. Total time elapsed = 0:04:39.422217


Token indices sequence length is longer than the specified maximum sequence length for this model (1259 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1690 > 512). Running this sequence through the model will result in indexing errors


row 1 finished at 2020-12-11 13:29:26.767254. Row time = 0:05:50.567341. Total time elapsed = 0:10:30.081412


Token indices sequence length is longer than the specified maximum sequence length for this model (4146 > 512). Running this sequence through the model will result in indexing errors


...

row 220 finished at 2020-12-15 14:16:12.861058. Row time = 0:13:28.853366. Total time elapsed = 4 days, 0:57:16.175216


Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


row 221 finished at 2020-12-15 14:18:42.467552. Row time = 0:02:29.606494. Total time elapsed = 4 days, 0:59:45.781710


Token indices sequence length is longer than the specified maximum sequence length for this model (2028 > 512). Running this sequence through the model will result in indexing errors


row 222 finished at 2020-12-15 14:22:06.009103. Row time = 0:03:23.541551. Total time elapsed = 4 days, 1:03:09.323261
